In [1]:
import json
import re

In [2]:
d_train= json.load(open('createData/multiwoz21/db/train_db.json'))

In [3]:
d_rest = json.load(open('createData/multiwoz21/db/restaurant_db.json'))

In [4]:
d_hotel = json.load(open('createData/multiwoz21/db/hotel_db.json'))

In [5]:
d_police = json.load(open('createData/multiwoz21/db/police_db.json'))

In [6]:
d_hosp = json.load(open('createData/multiwoz21/db/hospital_db.json'))

In [7]:
d_attr = json.load(open('createData/multiwoz21/db/attraction_db.json'))

In [8]:
d_taxi = [{
  "taxi_colors" : ["black","white","red","yellow","blue","grey"],
  "taxi_types":  ["toyota","skoda","bmw","honda","ford","audi","lexus","volvo","volkswagen","tesla"],
  "taxi_phone": ["^[0-9]{10}$"]
}]

In [9]:
entity_db_map = {'train':d_train, 'restaurant': d_rest, 'police': d_police, 'hospital': d_hosp, 'attraction': d_attr, 'taxi':d_taxi,'hotel':d_hotel}

In [10]:
d_data = json.load(open('createData/multiwoz21/delex2.json'))

In [11]:
def get_results(semi,ent):
    db = entity_db_map[ent]
    ret=[]
    for row in db:
#         print(row)
        match = True
        for k in semi.keys():
            if k not in row:
                continue
            # normalise
            val = row[k].lower().strip()
            semi[k] = semi[k].lower().strip()
            if k == 'name':
                val = val.replace('the', '')
                semi[k] = semi[k].replace('the', '')
                val = val.replace("b & b", "bed and breakfast")
                semi[k] = semi[k].replace("b & b", "bed and breakfast")
                val = val.replace("restaurant", "")
                semi[k] = semi[k].replace("restaurant", "")
                if "hotel" in val and 'gonville' not in val:
                    val = val.replace(" hotel", "")
                if "hotel" in semi[k] and 'gonville' not in semi[k]:
                        semi[k] = semi[k].replace("hotel", "")
                val = val.strip()
                semi[k] = semi[k].strip()
            if(val!=semi[k] and semi[k]!='not mentioned' and semi[k]!='dontcare' and semi[k]!='none' and semi[k]!=''):
                match=False
                break

        if(match):
            ret.append(row)
    
    if len(ret) == 0:
        if "leaveAt" in semi.keys() or "arriveBy" in semi.keys():
#             print(semi)
            ret1=[]
            ret2=[]
            for row in db:
#                 print(row)
                match = True
                for k in semi.keys():
                    if k not in row:
                        continue
                    row[k] = row[k].lower()
                    if(k == "arriveBy" and semi[k]!='not mentioned' and semi[k]!='dontcare'and semi[k]!='none' and semi[k]!=''):
                        if semi[k] == "afternoon" or semi[k] == "after lunch":
                            if int(row[k][:2])<=16:
                                match=True
                            else:
                                match=False
                                break
                        elif semi[k] == "morning":
                            if int(row[k][:2])<=11:
                                match=True
                            else:
                                match=False
                                break        
                        elif semi[k][0]!= ':' and int(semi[k].split(':')[0][-2:]) >= int(row[k].split(':')[0][-2:]):
                            match=True
                        else:
                            match=False
                            break
                    elif(k == "leaveAt" and semi[k]!='not mentioned' and semi[k]!='dontcare'and semi[k]!='none' and semi[k]!=''):
                        if semi[k] == "afternoon" or semi[k] == "after lunch":
                            if int(row[k][:2])<=16:
                                match=True
                            else:
                                match=False
                                break
                        elif semi[k] == "morning":
                            if int(row[k][:2])<=11:
                                match=True
                            else:
                                match=False
                                break
                
                        elif semi[k][0]!= ':' and int(semi[k].split(':')[0][-2:])-1 <= int(row[k].split(':')[0][-2:]):
                            match=True
                        else:
                            match=False
                            break        
                    elif(row[k]!=semi[k] and semi[k]!='not mentioned' and semi[k]!='dontcare'and semi[k]!='none' and semi[k]!=''):
                        match=False
                        break
             
                if match:
#                     if k=="arriveBy" or semi[k] == "afternoon" or semi[k] == "after lunch" or semi[k] == "morning":
#                         ret1.append(row)
#                     elif int(semi[k].split(':')[0][-2:]) == int(row[k].split(':')[0][-2:]):
#                         ret1.append(row)
#                     else:
#                         ret2.append(row)
                    ret.append(row)
                        
#             if len(ret1) != 0:
#                 ret.extend(ret1)
#             else:
#                 ret.extend(ret2)
                        
    return ret

In [12]:
def check_query_semi(semi):
    
    for k in semi.keys():
        if((semi[k]!='not mentioned') and semi[k]!='' and semi[k]!='none'):
            print(k)
            return True


In [13]:
for dial_k in d_data.keys():
    all_results = []
    all_queries = []
    goal = d_data[dial_k]['goal']
    topics_allowed = []
    # mark the topics that are mentioned in the goal
    for t in ['train', 'attraction', 'taxi', 'restaurant', 'hotel']:
        if goal[t]:
            topics_allowed.append(t)
    current_topic = ''
    # mark the topics are finished
    topics_done = {'train' : False, 'restaurant' : False, 'attraction' : False, 'hotel':False, 'taxi':False}
    
    # go through theconversation logs and add the query and result key to it
    for utt in d_data[dial_k]['log']:
        meta = utt['metadata']
        new_meta = utt['metadata']
        text = utt['text']
        if(meta=={}):
            continue
        found = False
        
        # add the queries and results key
        utt['results']=[]
        utt['queries']=[]
        possible_topic = []
        for k in meta.keys():
            if (k in topics_allowed and k!='bus' and check_query_semi(meta[k]['semi'])):
                if(topics_done[k] != True):
                    possible_topic.append(k)
        
        # determine the current topic i.e. if 2 topics are possible then it means the current topics will be changed
        if len(possible_topic) == 1:
            current_topic = possible_topic[0]
        elif len(possible_topic) == 2:
            if current_topic == possible_topic[0]:
                topics_done[current_topic] = True
                current_topic = possible_topic[1]
            else:
                topics_done[current_topic] = True
                current_topic = possible_topic[0]        
        elif len(possible_topic) >= 3:
            print('ERROR'*100)
            print(possible_topic)
            
#                 try:
        if current_topic!='train' and current_topic!='taxi' and current_topic != '':
#                 print(current_topic)
#                 print(meta[current_topic]['semi'])
                utt['results'].extend(get_results(meta[current_topic]['semi'],current_topic))
                q = meta[current_topic]['semi']
                for b in meta[current_topic]['book']:
                    if b != 'booked':
                        q[b] = meta[current_topic]['book'][b]
                utt['queries'].extend((current_topic,q))
                all_queries.append((current_topic,q))
#                         utt['queries'].extend((k,meta[k]['semi']))
#                         all_queries.append((k,meta[k]['semi']))
                all_results.append(utt['results'])
                found=True
        elif current_topic=='taxi':
                utt['results'].extend([])
                q = meta[current_topic]['semi']
                for b in meta[current_topic]['book']:
                    if b != 'booked':
                        q[b] = meta[current_topic]['book'][b]
                utt['queries'].extend((current_topic,q))
                all_queries.append((current_topic,q))
#                         utt['queries'].extend((k,meta[k]['semi']))
#                         all_queries.append((k,meta[k]['semi']))
                all_results.append(utt['results'])
                found=True
        elif current_topic == 'train':
                text = text.split(' ')
                for i in range(len(text)):
                    if text[i].find('arriv') != -1: 
                        for cand in text[i:i+5]:
                            if cand.find(":") != -1:
                                new_meta[current_topic]['semi']['arriveBy'] = cand
                                break

                    if text[i].find('leav') != -1 or text[i].find('depart') != -1:
                        for cand in text[i:i+5]:
                            if cand.find(":") != -1:
                                new_meta[current_topic]['semi']['leaveAt'] = cand
                                break

                    if text[i][:2] == 'TR' and text[i][2:].isdecimal():
                        new_meta[current_topic]['semi']['trainID'] = text[i]
                utt['results'].extend(get_results(new_meta[current_topic]['semi'],current_topic))
                q = meta[current_topic]['semi']
                for b in meta[current_topic]['book']:
                    if b != 'booked':
                        q[b] = meta[current_topic]['book'][b]
                utt['queries'].extend((current_topic,q))
                all_queries.append((current_topic,q))
#                         utt['queries'].extend((k,meta[k]['semi']))
#                         all_queries.append((k,meta[k]['semi']))
                all_results.append(utt['results'])
                found=True

    d_data[dial_k]['all_queries'] = all_queries
    d_data[dial_k]['all_results'] = all_results
    print('Adding ', len(all_queries), 'queries')
    print('Adding ', sum(map(lambda x: len(x),all_results)), 'results')

pricerange
parking
parking
parking
parking
Adding  5 queries
Adding  5 results
Adding  0 queries
Adding  0 results
area
area
name
name
name
destination
name
destination
name
destination
name
destination
name
destination
Adding  9 queries
Adding  49 results
destination
destination
destination
destination
stars
destination
name
destination
name
destination
name
destination
Adding  8 queries
Adding  1293 results
destination
leaveAt
leaveAt
leaveAt
Adding  4 queries
Adding  0 results
Adding  0 queries
Adding  0 results
area
area
name
area
area
area
departure
area
leaveAt
area
leaveAt
area
leaveAt
Adding  9 queries
Adding  146 results
leaveAt
destination
destination
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
Adding  11 queries
Adding  82 results
area
pricerange
area
food
area
food
name
food
name
food
name
Adding  6 queries
Adding  80 results
Adding  0 queries
Adding  0 results
day
leaveAt
leaveAt
leaveAt
type
leaveAt
type
leaveAt
Addin

food
destination
food
destination
Adding  9 queries
Adding  113 results
food
food
food
food
leaveAt
food
leaveAt
food
leaveAt
Adding  6 queries
Adding  19 results
name
name
pricerange
name
food
name
food
name
food
name
food
name
destination
food
name
destination
food
name
Adding  9 queries
Adding  17 results
name
name
name
name
name
leaveAt
name
leaveAt
name
leaveAt
name
leaveAt
name
leaveAt
food
name
leaveAt
food
name
leaveAt
food
name
leaveAt
food
name
Adding  13 queries
Adding  13 results
food
food
food
food
food
Adding  5 queries
Adding  6 results
area
area
area
area
name
name
name
type
name
type
name
type
name
type
Adding  10 queries
Adding  26 results
day
destination
destination
destination
destination
Adding  5 queries
Adding  358 results
destination
leaveAt
leaveAt
leaveAt
leaveAt
leaveAt
leaveAt
leaveAt
Adding  8 queries
Adding  131 results
parking
area
area
area
area
area
food
area
food
area
leaveAt
food
area
leaveAt
food
area
Adding  10 queries
Adding  39 results
name
name
n

leaveAt
leaveAt
leaveAt
type
leaveAt
type
leaveAt
type
leaveAt
Adding  11 queries
Adding  576 results
day
day
day
internet
day
area
day
area
day
name
day
name
day
name
day
name
destination
name
destination
name
destination
name
destination
Adding  13 queries
Adding  462 results
parking
area
area
area
area
area
area
Adding  7 queries
Adding  48 results
food
food
food
food
Adding  4 queries
Adding  0 results
day
destination
leaveAt
leaveAt
pricerange
leaveAt
pricerange
leaveAt
parking
leaveAt
parking
leaveAt
name
leaveAt
name
leaveAt
name
leaveAt
Adding  11 queries
Adding  487 results
type
type
food
type
food
type
food
type
Adding  5 queries
Adding  10 results
name
name
name
name
name
parking
name
parking
name
parking
name
parking
name
parking
name
parking
Adding  10 queries
Adding  69 results
destination
leaveAt
leaveAt
leaveAt
leaveAt
Adding  5 queries
Adding  1470 results
food
food
food
food
food
Adding  5 queries
Adding  4 results
type
type
type
type
destination
type
destination
type

name
destination
name
destination
name
destination
name
destination
name
destination
Adding  9 queries
Adding  649 results
food
food
food
food
area
food
type
food
type
food
type
Adding  7 queries
Adding  104 results
destination
leaveAt
leaveAt
name
leaveAt
name
leaveAt
name
leaveAt
Adding  6 queries
Adding  83 results
internet
parking
parking
parking
parking
parking
Adding  6 queries
Adding  49 results
area
area
area
name
name
destination
name
leaveAt
name
leaveAt
name
leaveAt
name
leaveAt
Adding  9 queries
Adding  264 results
pricerange
area
area
area
area
type
area
type
area
type
area
type
Adding  8 queries
Adding  92 results
Adding  0 queries
Adding  0 results
name
name
name
name
type
name
type
leaveAt
name
type
leaveAt
name
type
leaveAt
name
type
Adding  8 queries
Adding  5 results
food
food
food
food
Adding  4 queries
Adding  4 results
area
area
area
area
area
area
destination
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
Adding  10 queries
Adding  97 results
stars
parking
p

destination
destination
destination
destination
Adding  6 queries
Adding  502 results
Adding  0 queries
Adding  0 results
pricerange
food
food
food
food
day
food
destination
food
destination
food
destination
food
destination
food
destination
Adding  10 queries
Adding  225 results
type
type
type
type
Adding  4 queries
Adding  0 results
area
pricerange
pricerange
pricerange
area
pricerange
area
pricerange
area
pricerange
area
Adding  7 queries
Adding  279 results
name
name
name
destination
name
destination
name
destination
name
destination
name
destination
Adding  7 queries
Adding  376 results
parking
parking
parking
parking
day
parking
destination
parking
destination
parking
destination
parking
destination
Adding  8 queries
Adding  227 results
food
food
food
food
food
area
food
area
food
area
Adding  7 queries
Adding  8 results
pricerange
food
food
food
food
type
food
type
food
type
Adding  7 queries
Adding  46 results
type
type
food
type
food
type
food
type
food
type
food
type
Adding  

...

name
leaveAt
name
leaveAt
name
leaveAt
Adding  13 queries
Adding  111 results
destination
destination
destination
pricerange
destination
parking
destination
name
destination
name
destination
name
destination
Adding  8 queries
Adding  153 results
food
food
food
food
food
Adding  5 queries
Adding  5 results
food
food
food
food
Adding  4 queries
Adding  3 results
parking
area
area
area
area
destination
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
Adding  9 queries
Adding  31 results
destination
leaveAt
leaveAt
Adding  3 queries
Adding  0 results
name
food
name
food
name
food
name
food
name
food
name
food
name
food
name
food
name
food
name
leaveAt
food
name
leaveAt
food
name
leaveAt
food
name
Adding  13 queries
Adding  14 results
name
name
name
area
name
area
name
area
Adding  5 queries
Adding  134 results
departure
destination
leaveAt
leaveAt
leaveAt
Adding  5 queries
Adding  98 results
pricerange
pricerange
pricerange
pricerange
pricerange
Adding  5 queries
Adding  37 results
food

food
leaveAt
food
leaveAt
food
leaveAt
Adding  7 queries
Adding  1123 results
area
area
area
name
name
name
name
name
name
name
name
name
name
name
name
name
Adding  10 queries
Adding  23 results
destination
destination
destination
destination
destination
destination
Adding  6 queries
Adding  0 results
area
area
area
area
Adding  4 queries
Adding  8 results
area
area
name
name
destination
name
destination
name
destination
name
destination
Adding  7 queries
Adding  96 results
food
food
food
food
food
destination
food
destination
food
destination
food
destination
Adding  8 queries
Adding  79 results
area
area
name
name
name
day
name
destination
Adding  6 queries
Adding  311 results
area
area
area
name
food
name
food
name
food
name
food
name
food
name
Adding  9 queries
Adding  14 results
food
food
food
Adding  3 queries
Adding  0 results
area
area
area
name
name
name
name
name
name
type
name
type
leaveAt
name
type
leaveAt
name
type
Adding  12 queries
Adding  11 results
day
leaveAt
leaveAt

type
leaveAt
type
leaveAt
type
leaveAt
type
leaveAt
Adding  11 queries
Adding  74 results
departure
destination
leaveAt
leaveAt
leaveAt
Adding  5 queries
Adding  98 results
pricerange
area
name
name
day
name
destination
name
leaveAt
name
leaveAt
name
leaveAt
Adding  8 queries
Adding  437 results
Adding  0 queries
Adding  0 results
departure
destination
destination
destination
destination
Adding  5 queries
Adding  94 results
parking
area
area
name
name
day
name
destination
name
destination
name
destination
Adding  8 queries
Adding  444 results
food
food
food
food
food
food
Adding  6 queries
Adding  12 results
type
type
type
area
type
name
type
name
type
Adding  6 queries
Adding  4 results
name
name
name
name
Adding  4 queries
Adding  4 results
area
food
food
food
food
food
pricerange
food
name
food
name
destination
food
name
leaveAt
food
name
destination
food
name
destination
food
name
destination
food
name
Adding  13 queries
Adding  87 results
leaveAt
leaveAt
leaveAt
destination
destin

leaveAt
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
area
leaveAt
Adding  9 queries
Adding  1029 results
destination
destination
destination
internet
destination
Adding  4 queries
Adding  166 results
stars
area
area
area
area
name
name
day
name
leaveAt
name
leaveAt
name
leaveAt
Adding  10 queries
Adding  27 results
food
food
food
food
food
food
area
food
area
food
name
destination
food
name
destination
food
name
destination
food
name
Adding  11 queries
Adding  92 results
Adding  0 queries
Adding  0 results
departure
departure
destination
Adding  3 queries
Adding  0 results
area
food
food
food
food
name
destination
food
name
destination
food
name
Adding  7 queries
Adding  17 results
type
type
type
type
day
type
leaveAt
type
leaveAt
type
leaveAt
Adding  7 queries
Adding  44 results
pricerange
food
food
Adding  3 queries
Adding  11 results
leaveAt
leaveAt
leaveAt
leaveAt
type
leaveAt
type
leaveAt
type
leaveAt
Adding  7 queries
Adding  1378 results
area
area
are

name
destination
name
name
Adding  10 queries
Adding  32 results
destination
destination
destination
destination
parking
destination
parking
destination
name
destination
Adding  7 queries
Adding  84 results
parking
parking
parking
name
name
name
Adding  6 queries
Adding  23 results
day
destination
destination
destination
type
destination
type
destination
Adding  6 queries
Adding  428 results
food
pricerange
food
food
food
food
Adding  6 queries
Adding  47 results
food
food
food
food
food
Adding  5 queries
Adding  13 results
food
food
food
destination
food
destination
food
destination
Adding  5 queries
Adding  8 results
leaveAt
leaveAt
Adding  2 queries
Adding  0 results
destination
leaveAt
leaveAt
parking
leaveAt
parking
leaveAt
name
leaveAt
name
leaveAt
Adding  7 queries
Adding  124 results
type
type
type
destination
type
destination
type
destination
type
destination
Adding  6 queries
Adding  648 results
food
food
food
food
food
area
leaveAt
food
area
leaveAt
food
area
Adding  7 queri

In [14]:
# for dial_k in d_data.keys():
#     all_results = []
#     all_queries = []
#     for utt in d_data[dial_k]['log']:
#         meta = utt['metadata']
#         new_meta = utt['metadata']
#         text = utt['text']
#         if(meta=={}):
#             continue
#         found = False
#         utt['results']=[]
#         utt['queries']=[]
#         utt['modified_queries']=[]
#         for k in meta.keys():
#             if(k!='taxi' and k!='bus' and check_query_semi(meta[k]['semi'])):
# #                 try:
#                 if k!='train':
#                     if((k,meta[k]['semi']) not in all_queries):
#                         utt['results'].extend(get_results(meta[k]['semi'],k))
#                         utt['queries'].extend((k,meta[k]['semi']))
#                         utt['modified_queries'].extend((k,new_meta[k]['semi']))
#                         all_queries.append((k,meta[k]['semi']))
#                         all_results.append(utt['results'])
#                         found=True
#                 else:
#                     if((k,meta[k]['semi']) not in all_queries):
# #                         text = text.split(' ')
# #                         for i in range(len(text)):
# #                             if text[i].find('arriv') != -1: 
# #                                 for cand in text[i:i+5]:
# #                                     if cand.find(":") != -1:
# #                                         new_meta[k]['semi']['arriveBy'] = cand
# #                                         break
                        
# #                             if text[i].find('leav') != -1 or text[i].find('depart') != -1:
# #                                 for cand in text[i:i+5]:
# #                                     if cand.find(":") != -1:
# #                                         new_meta[k]['semi']['leaveAt'] = cand
# #                                         break
                                    
# #                             if text[i][:2] == 'TR' and text[i][2:].isdecimal():
# #                                 new_meta[k]['semi']['trainID'] = text[i]
#                         utt['results'].extend(get_results(new_meta[k]['semi'],k))
#                         utt['queries'].extend((k,meta[k]['semi']))
#                         utt['modified_queries'].extend((k,new_meta[k]['semi']))
#                         all_queries.append((k,meta[k]['semi']))
#                         all_results.append(utt['results'])
#                         found=True

#     d_data[dial_k]['all_queries'] = all_queries
#     d_data[dial_k]['all_results'] = all_results
#     print('Adding ', len(all_queries), 'queries')
#     print('Adding ', sum(map(lambda x: len(x),all_results)), 'results')

In [15]:
json.dump(d_data, open('createData/multiwoz21/delex_query_results.json','w'))